In [ ]:
#default_exp update

In [ ]:
#hide
import pickle, os, json

os.environ['DATABASE_TABLE_NAME'] = 'product-table-dev-manual'
os.environ['REGION'] = 'ap-southeast-1'
os.environ['INVENTORY_BUCKET_NAME'] = 'product-bucket-dev-manual'
os.environ['INPUT_BUCKET_NAME'] = 'input-product-bucket-dev-manual'
os.environ['DAX_ENDPOINT'] = 'longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111'
os.environ['LINEKEY'] = '2uAfV4AoYglUGmKTAk2xNOm0aV2Ufgh1BQPvQl9vJd4'
os.environ['INTCOLS'] = json.dumps(['cprcode', 'iprcode', 'oprcode', 'sellingPrice'])
os.environ['BRANCH'] = 'dev-manual'
REGION = 'ap-southeast-1'

# Update
update the database

In [ ]:
#export
from s3bz.s3bz import S3
from nicHelper.wrappers import add_method, add_class_method, add_static_method
from nicHelper.dictUtil import stripDict, printDict, hashDict, saveStringToFile, loadStringFromFile, saveDictToFile, loadDictFromFile
from nicHelper.exception import errorString
from dict_hash import dict_hash, sha256
from base64 import b64encode, b64decode
from dataclasses_json import dataclass_json, Undefined, CatchAll
from dataclasses_jsonschema import JsonSchemaMixin
from dataclasses import dataclass
from jsonschema import validate
from typing import List
from datetime import datetime
from nicHelper.schema import validateUrl
import pandas as pd
import os, logging, json

In [ ]:
#hide
from villaProductDatabase.database import ProductDatabase
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
#export
import os
DBHASHLOCATION = '/tmp/database.hash'
DBCACHELOCATION = '/tmp/database.cache'
DATABASE_TABLE_NAME = os.environ.get('DATABASE_TABLE_NAME')
INVENTORY_BUCKET_NAME = os.environ.get('INVENTORY_BUCKET_NAME')
INPUT_BUCKET_NAME = os.environ.get('INPUT_BUCKET_NAME')
REGION = os.environ.get('REGION') or 'ap-southeast-1'
ACCESS_KEY_ID = os.environ.get('USER') or None
SECRET_ACCESS_KEY = os.environ.get('PW') or None
LINEKEY= os.environ.get('LINEKEY')
BRANCH = os.environ.get('BRANCH')
VALUEUPDATESCHEMAURL = f'https://raw.githubusercontent.com/thanakijwanavit/villaMasterSchema/{BRANCH}/product/product.yaml'
  
try:
  DAX_ENDPOINT = os.environ['DAX_ENDPOINT']
  print(DAX_ENDPOINT)
except KeyError as e:
  print(f'dax endpoint missing {e}')

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


In [ ]:
#export
class Updater:
  pass

In [ ]:
class Tester( Updater, ProductDatabase):
  class Meta:
    table_name = os.environ['DATABASE_TABLE_NAME']
    region = os.environ['REGION']
    billing_mode='PAY_PER_REQUEST'
    dax_read_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
    dax_write_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
  pass

## Update with dict

In [ ]:
#export
@add_class_method(Updater)
def updateWithDict(cls, originalObject:Updater, inputDict:dict ):
  data = originalObject.data
  data.update(inputDict)
  return cls.fromDict(data)

In [ ]:
# test
item = next(Tester.query(171670))
print(f'engname is {item.data.get("pr_engname")}')

updatedItem = Tester.updateWithDict(item,{'pr_engname':'testName'})
assert updatedItem.data.get('pr_engname') == 'testName'
print(updatedItem.data.get('pr_engname'))

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


engname is JIRAPAT YOUNG KALE 200 G.
testName


## update with list of dict

In [ ]:
#hide
from dataclasses_json import dataclass_json, Undefined, CatchAll
from dataclasses_jsonschema import JsonSchemaMixin
from typing import Optional
import yaml
@dataclass_json(undefined=Undefined.INCLUDE)
@dataclass
class Product(JsonSchemaMixin):
  iprcode: int
  cprcode: int
  oprcode: Optional[int]
  pr_dpcode: Optional[str]
  pr_barcode: Optional[str]
  pr_barcode2: Optional[str]
  pr_sucode1: Optional[str]
  pr_suref3: Optional[str]
  pr_sa_method: Optional[str]
  sellingPrice: Optional[int]
#   data: CatchAll
@dataclass_json
@dataclass
class ValueUpdate(JsonSchemaMixin):
  items: List[Product]
# print(yaml.dump(Product.json_schema(), indent=2, sort_keys=True))
print(yaml.dump(ValueUpdate.json_schema(), indent=2, sort_keys=True))

$schema: http://json-schema.org/draft-06/schema#
definitions:
  Product:
    description: 'Product(iprcode: int, cprcode: int, oprcode: Union[int, NoneType],
      pr_dpcode: Union[str, NoneType], pr_barcode: Union[str, NoneType], pr_barcode2:
      Union[str, NoneType], pr_sucode1: Union[str, NoneType], pr_suref3: Union[str,
      NoneType], pr_sa_method: Union[str, NoneType], sellingPrice: Union[int, NoneType])'
    properties:
      cprcode:
        type: integer
      iprcode:
        type: integer
      oprcode:
        type: integer
      pr_barcode:
        type: string
      pr_barcode2:
        type: string
      pr_dpcode:
        type: string
      pr_sa_method:
        type: string
      pr_sucode1:
        type: string
      pr_suref3:
        type: string
      sellingPrice:
        type: integer
    required:
    - iprcode
    - cprcode
    type: object
description: 'ValueUpdate(items: List[__main__.Product])'
properties:
  items:
    items:
      $ref: '#/definitions/Pr

In [ ]:
#export
class ValueUpdate2:
   #### divide work into chunks
  @staticmethod
  def chunks(l, n): return [l[x: x+n] for x in range(0, len(l), n)]
  ##### plot time elapsed for debugging
  @staticmethod
  def getTimeElasped(t0): return (datetime.now()-t0).total_seconds() * 1000

  t0 = datetime.now()

  ### validate input
  @staticmethod
  def validateInput(input_):
    return validateUrl(VALUEUPDATESCHEMAURL, 
                       input_, format_='yaml')

In [ ]:
sampleProducts = {'items':[{ 'cprcode': 171670, 'iprcode': 171670, 'oprcode': 171670, 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1', 'pr_barcode': '123', 'pr_barcode2': '245'}]}
# t0 = datetime.now()
# ValueUpdate2.chunks(sampleProducts, 500)
# ValueUpdate2.getTimeElasped(t0)
# schema = requests.get(VALUEUPDATESCHEMAURL).json()
# validate(sampleProducts, schema)
ValueUpdate2.validateInput(sampleProducts).items

[{'cprcode': 171670,
  'iprcode': 171670,
  'oprcode': 171670,
  'ordertype': 'Y',
  'pr_abb': 'JIRAPAT YOUNG KALE 2',
  'pr_active': 'Y',
  'pr_cgcode': '05',
  'pr_code': '0171670',
  'pr_dpcode': '19',
  'pr_engname': 'JIRAAT YOUNG KALE 200 G.',
  'pr_ggcode': '057',
  'pr_market': 'JIRAPAT ยอดคะน้า 200 G.',
  'pr_name': 'JIRAPAT ยอดคะน้า 200 G.',
  'pr_puqty': '1',
  'pr_sa_method': '1',
  'pr_sucode1': 'CM845',
  'pr_suref3': 'A',
  'prtype': 'I',
  'psqty': '1',
  'pstype': '1',
  'pr_barcode': '123',
  'pr_barcode2': '245'}]

In [ ]:
#export
@add_static_method(ValueUpdate2)
def saveBatchToDynamodb(productBatch:list, dbClass:Updater, db:pd.DataFrame, itemsUpdated:dict):
  def checkIfInDb(db:pd.DataFrame, iprcode:int, cprcode:int)->dbClass:
    '''check if product is in the database, if not, create an empty class with the product code''' 
    if 'cprcode' in db.columns:
      incumbentDb = db[db['cprcode']==cprcode] ## filter for the cprcode
      if incumbentDb.empty:
        incumbentSeries = pd.Series() ## if empty, create an empty series
      else:
        incumbentSeries = incumbentDb.iloc[0] ## if not empty, pull the series
        
      #### create object from series or new object if series is empty ####
      if incumbentSeries.any:  incumbentItem = dbClass.fromSeries(incumbentSeries)
      else: incumbentItem = dbClass.fromDict({'iprcode': iprcode, 'cprcode': cprcode})
        
    #### if cprcode doesnt exist, create empty object ####
    else: incumbentItem = dbClass.fromDict({'iprcode': iprcode, 'cprcode': cprcode})
    return incumbentItem
      
  ###### main ######
  with dbClass.batch_write() as batch:
    # loop through each product
    for product in productBatch:
      iprcode = product['iprcode']
      cprcode = product['cprcode']
      incumbentItem = checkIfInDb(db, iprcode, cprcode) ### check if product in db or return empty obj

      ##### make a copy of original data
      originalData = incumbentItem.data.copy()
      ###### update data
      updatedData = dbClass.updateWithDict(incumbentItem, product)

      logging.info(f'incumbentBr is {incumbentItem.iprcode}\n, prcode is {iprcode}')

      # check for difference
      try:
        if updatedData.data != originalData:
          logging.info(f'product {iprcode} has changed from \n{originalData} \n{updatedData.data}')
          batch.save(updatedData)
          itemsUpdated['success'] += 1
        else:
          logging.info(f'no change for {iprcode}')
          itemsUpdated['skipped'] += 1
      except Exception as e:
        itemsUpdated['failure'] += 1
        itemsUpdated['failureMessage'].append(e)

    # log time taken
#     itemsUpdated['timetaken(ms)'] = (datetime.now()- t0).total_seconds()*1000

In [ ]:
#export
@add_static_method(ValueUpdate2)
def saveBatchToDynamodb2(productBatch:list, dbClass:Updater, db:pd.DataFrame, itemsUpdated:dict):
  def checkIfInDb(db:pd.DataFrame, iprcode:int, cprcode:int)->dbClass:
    '''check if product is in the database, if not, create an empty class with the product code''' 
    incumbentItem =  next(dbClass.query(hash_key = iprcode, range_key_condition=dbClass.cprcode == cprcode), dbClass(iprcode=iprcode, cprcode=cprcode, data = {}))
    return incumbentItem
      
  ###### main ######
  with dbClass.batch_write() as batch:
    # loop through each product
    for product in productBatch:
      iprcode = product['iprcode']
      cprcode = product['cprcode']
      incumbentItem = checkIfInDb(db, iprcode, cprcode) ### check if product in db or return empty obj

      ##### make a copy of original data
      originalData = incumbentItem.data.copy()
      ###### update data
      updatedData = dbClass.updateWithDict(incumbentItem, product)
      
      # check for difference
      try:
        if updatedData.data != originalData:
          logging.info(f'product {iprcode} has changed from \n{originalData} \n{updatedData.data}')
          batch.save(updatedData)
          itemsUpdated['success'] += 1
        else:
          logging.info(f'no change for {iprcode}')
          itemsUpdated['skipped'] += 1
          
      except Exception as e:
        itemsUpdated['failure'] += 1
        itemsUpdated['failureMessage'].append(e)



In [ ]:
#export
@add_class_method(Updater)
def valueUpdate2(cls, inputs:List[dict]):
    ''' 
      check for difference and batch update the changes in product data
    '''
    #### setup #####
    itemsUpdated = {'success':0, 'failure': 0, 'skipped': 0 ,'failureMessage':[], 'timetaken(ms)': 0}
    t0 = datetime.now()
    
    ##### main #####
    
    ######validate input ###########
    products = ValueUpdate2.validateInput(inputs).items
    print(f'products are {products[0]}')
    print(f'time taken for validation  {ValueUpdate2.getTimeElasped(t0)} ms')
    
    ######dividing input into batch of 500###########
    productBatches = ValueUpdate2.chunks(products, 500)
    print(f'divided into chunks {ValueUpdate2.getTimeElasped(t0)} ms')
    
    ######try to load from cache########
    db = cls.loadFromCache().fillna('none')
    print(f'get all from s3 {ValueUpdate2.getTimeElasped(t0)} ms')
    
    ###### save to dynamodb ########
    for productBatch in productBatches:
      ValueUpdate2.saveBatchToDynamodb2(productBatch, cls, db, itemsUpdated)
    itemsUpdated['timetaken(ms)'] = (datetime.now()- t0).total_seconds()*1000
    return itemsUpdated
  

In [ ]:
%%time
# sampleProducts = {'items':[{ 'cprcode': 171670, 'iprcode': 171670, 'oprcode': 171670, 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}]}
# products = ValueUpdate.from_dict({'items':sampleProducts}).to_dict()
Tester.valueUpdate2(sampleProducts)
# Tester.saveRemoteCache()

INFO:root:using accelerate endpoint
INFO:root:using accelerate endpoint


products are [{'cprcode': 171670, 'iprcode': 171670, 'oprcode': 171670, 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1', 'pr_barcode': '123', 'pr_barcode2': '245'}]
time taken for validation  32.619 ms
divided into chunks 32.673 ms
loading hashkey allData-hash
loaded hash is 660eb195aed7c244a8c510463516d1d6df3e90f9
data is still in sync, using local cache


INFO:root:no change for 171670


get all from s3 631.664 ms
CPU times: user 496 ms, sys: 71.3 ms, total: 568 ms
Wall time: 662 ms


{'success': 0,
 'failure': 0,
 'skipped': 1,
 'failureMessage': [],
 'timetaken(ms)': 640.167}

In [ ]:
db = Tester.loadFromCache()
db

INFO:root:using accelerate endpoint
INFO:root:using accelerate endpoint
INFO:root:using accelerate endpoint


loading hashkey allData-hash
loaded hash is fef8a7b62e18cd6e4cd7ad89b90be9d917d5a657
remote hash is not the same, load remote cache


,cprcode,iprcode,oprcode,ordertype,pr_abb,pr_active,pr_cgcode,pr_code,pr_dpcode,pr_engname,...,hema_name_th,pr_name_en,pr_name_th,sort_weight,pr_barcode,pr_barcode2,master_online,SaleMode_unit,ba_nprice,sort_villa_sku
0,225407,225407,225407,Y,MOKU,Y,87,225407,21.0,KONJAC LINGUINI,...,บุกเส้นแบน ตราโมกุ,KONJAC LINGUINI,บุกเส้นแบน ตราโมกุ,0.0,NaN,NaN,None,None,None,NaN
1,241101,241101,241101,Y,BUMILGOCHUJANG,Y,06,0241101,8.0,BUMILGOCHUJANG,...,none,BUMILGOCHUJANG,บูมิลโคชูจังซอสเผ้ดเกาหลี,0.0,NaN,NaN,None,None,None,NaN
2,190100,190100,190100,Y,WHITE SHIMEJI,Y,5,190100,19.0,WHITE SHIMEJI,...,QUALITY VEGETABLE เห็ดชิเมจิขาว,WHITE SHIMEJI,QUALITY VEGETABLE เห็ดชิเมจิขาว,0.0,NaN,NaN,None,None,None,NaN
3,62644,62644,62644,Y,STUTE APRICOT JAM430,Y,7,62644,8.0,STUTE APRICOT JAM 430 G.,...,Apricot Jam,STUTE APRICOT JAM 430 G.,STUTE APRICOT JAM 430 G.,0.0,NaN,NaN,None,None,None,NaN
4,192167,192167,192167,Y,**COLGATE TOTAL CHAR,Y,10,192167,9.0,**COLGATE TOTAL CHARCOAL DEEP CLEAN 150G,...,Total Charcoal Deep Clean,**COLGATE TOTAL CHARCOAL DEEP CLEAN 150G,COGATE ยสฟ.โททอลชาร์โคลดีพคลีน 150กรัม,0.0,NaN,NaN,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43797,166008,166008,166008,Y,RIBBON 180 M. 21,Y,2,166008,25.0,RIBBON 180 M. 21,...,RIBBON 180 M. 21,RIBBON 180 M. 21,ริบบิ้นผ้า 180ม.21 มิล,0.0,NaN,NaN,None,None,None,NaN
43798,76931,76931,76931,Y,ST. REMY BRANDY 70CL,Y,16,76931,1.0,ST. REMY BRANDY 70CL.,...,Brandy,ST. REMY BRANDY 70CL.,ST. REMY BRANDY 70CL.,0.0,NaN,NaN,None,None,None,NaN
43799,203391,203391,203391,Y,CH TERREFORT-QUANCA,Y,10,203391,1.0,CH TERREFORT-QUANCARD,...,Bordeaux,CH TERREFORT-QUANCARD,CH TERREFORT-QUANCARD,0.0,NaN,NaN,None,None,None,NaN
43800,205603,205603,205603,Y,19 GRAIN WHEAT LOAF,Y,15,205603,19.0,19 GRAIN WHEAT LOAF,...,19 Grain Wheat Loaf,19 GRAIN WHEAT LOAF,ขนมปังโฮลวีตธัญพืช 19 ชนิด,0.0,NaN,NaN,None,None,None,NaN


In [ ]:
next(Tester.query(hash_key = 9, range_key_condition=Tester.cprcode == 9))

ProductDatabase Object
cprcode 9
iprcode 9
oprcode 9
ordertype Y
pr_abb J. WALKER RED 70 CL.
pr_active Y
pr_cgcode 01
pr_code 0000009
pr_dpcode 01
pr_engname JOHNNIE WALKER RED 70 CL.
pr_ggcode 001
pr_market J. WALKER RED 70 CL
pr_name JOHNNIE WALKER RED 70 CL
pr_puqty 12.00
pr_sa_method 1
pr_sucode1 1782
pr_suref3 A
prtype I
psqty 1
pstype 1
pr_country_th nan
pr_country_en United Kingdom
pr_keyword_th nan
pr_keyword_en nan
pr_filter_th Alcohol,Spirits  Liqueurs
pr_filter_en Alcohol,Spirits  Liqueurs
online_category_l1_th เบียร์, ไวน์และสุรา
online_category_l1_en Beer Wine & Spirits
online_category_l2_th เหล้า
online_category_l2_en Spirits & Liqueurs
online_category_l3_th nan
online_category_l3_en nan
villa_category_l1_en Dry Grocery
villa_category_l2_en Alcohol & Beverage
villa_category_l3_en Whisky
villa_category_l4_en Scoth Whisky
content_en 0000009 Johnnie Walker Red 700ml
content_th JOHNNIE WALKER RED 700ml
hema_brand_th Johnnie Walker
hema_brand_en Johnnie Walker
hema_sizedesc 70